In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")
%matplotlib inline

In [2]:
df = sns.load_dataset("diamonds")
df

carat        cut color clarity  depth  table  price     x     y     z
0       0.23      Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1       0.21    Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2       0.23       Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3       0.29    Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4       0.31       Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75
...      ...        ...   ...     ...    ...    ...    ...   ...   ...   ...
53935   0.72      Ideal     D     SI1   60.8   57.0   2757  5.75  5.76  3.50
53936   0.72       Good     D     SI1   63.1   55.0   2757  5.69  5.75  3.61
53937   0.70  Very Good     D     SI1   62.8   60.0   2757  5.66  5.68  3.56
53938   0.86    Premium     H     SI2   61.0   58.0   2757  6.15  6.12  3.74
53939   0.75      Ideal     D     SI2   62.2   55.0   2757  5.83  5.87  3.64

[53940 rows x 10 columns]

In [3]:
label_name = "price"

In [4]:
X_raw = df.drop(label_name, axis=1)
y = df[label_name]

X_raw.shape, y.shape

((53940, 9), (53940,))

In [5]:
X = pd.get_dummies(X_raw)
X

carat  depth  table     x     y     z  cut_Ideal  cut_Premium  \
0       0.23   61.5   55.0  3.95  3.98  2.43          1            0   
1       0.21   59.8   61.0  3.89  3.84  2.31          0            1   
2       0.23   56.9   65.0  4.05  4.07  2.31          0            0   
3       0.29   62.4   58.0  4.20  4.23  2.63          0            1   
4       0.31   63.3   58.0  4.34  4.35  2.75          0            0   
...      ...    ...    ...   ...   ...   ...        ...          ...   
53935   0.72   60.8   57.0  5.75  5.76  3.50          1            0   
53936   0.72   63.1   55.0  5.69  5.75  3.61          0            0   
53937   0.70   62.8   60.0  5.66  5.68  3.56          0            0   
53938   0.86   61.0   58.0  6.15  6.12  3.74          0            1   
53939   0.75   62.2   55.0  5.83  5.87  3.64          1            0   

       cut_Very Good  cut_Good  ...  color_I  color_J  clarity_IF  \
0                  0         0  ...        0        0           0   
1                  0         0  ...        0        0           0   
2                  0         1  ...        0        0           0   
3                  0         0  ...        1        0           0   
4                  0         1  ...        0        1           0   
...              ...       ...  ...      ...      ...         ...   
53935              0         0  ...        0        0           0   
53936              0         1  ...        0        0           0   
53937              1         0  ...        0        0           0   
53938              0         0  ...        0        0           0   
53939              0         0  ...        0        0           0   

       clarity_VVS1  clarity_VVS2  clarity_VS1  clarity_VS2  clarity_SI1  \
0                 0             0            0            0            0   
1                 0             0            0            0            1   
2                 0             0            1            0            0   
3                 0             0            0            1            0   
4                 0             0            0            0            0   
...             ...           ...          ...          ...          ...   
53935             0             0            0            0            1   
53936             0             0            0            0            1   
53937             0             0            0            0            1   
53938             0             0            0            0            0   
53939             0             0            0            0            0   

       clarity_SI2  clarity_I1  
0                1           0  
1                0           0  
2                0           0  
3                0           0  
4                1           0  
...            ...         ...  
53935            0           0  
53936            0           0  
53937            0           0  
53938            1           0  
53939            1           0  

[53940 rows x 26 columns]

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X.values, y, test_size=0.33, random_state=42)

* [예제로 배우는 파이토치(PyTorch) — 파이토치 한국어 튜토리얼 (PyTorch tutorials in Korean)](https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html)
* [Linear — PyTorch documentation](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(in_features=X_train.shape[1], out_features=1) 

    def forward(self, x):
        return self.linear(x)

In [23]:
model = MultivariateLinearRegressionModel()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

model.parameters

<bound method Module.parameters of MultivariateLinearRegressionModel(
  (linear): Linear(in_features=26, out_features=1, bias=True)
)>

In [27]:
# y_train 의 차원을 계산을 위해 증가시킴
np.expand_dims(y_train, axis=1)

array([[2788],
       [ 612],
       [ 477],
       ...,
       [1014],
       [2871],
       [6320]])

In [28]:
# torch 로 unsqueeze
torch.FloatTensor(y_train).unsqueeze(1)

tensor([[2788.],
        [ 612.],
        [ 477.],
        ...,
        [1014.],
        [2871.],
        [6320.]])

In [31]:
nb_epochs = 100
for epoch in range(nb_epochs+1):

    prediction = model(torch.FloatTensor(X_train))
    cost = F.mse_loss(prediction, torch.FloatTensor(np.expand_dims(y_train, axis=1)))
    optimizer.zero_grad()
    # gradient
    cost.backward()
    # Weight and bias update
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/100 Cost: 15783279.000000
Epoch   10/100 Cost: 15767647.000000
Epoch   20/100 Cost: 15752033.000000
Epoch   30/100 Cost: 15736441.000000
Epoch   40/100 Cost: 15720868.000000
Epoch   50/100 Cost: 15705314.000000
Epoch   60/100 Cost: 15689782.000000
Epoch   70/100 Cost: 15674269.000000
Epoch   80/100 Cost: 15658779.000000
Epoch   90/100 Cost: 15643305.000000
Epoch  100/100 Cost: 15627852.000000
